# MicroStrategy Embedding - How to embed an interactive Dossier during your analysis

### Python, JavaScript and HTML with Jupyter Magic commands

Example: 

Demonstrate how to use Python to navigate and find the published Dossier we are interested in and display it inside Jupyter Notebook. This will give us an interactive Dossier for data analysis to better understand the data published by the business. This will be our exploratory step before loading the cube(s) into DataFrames. Here we get to understand the analysis done by the business, and how they like to analyse their own data to gain a better intuition. 

In this example I will show you how to use pure code all within Jupyter Notebook to accomplish this without having to go to MicroStrategy Web or needing to right-click to get object properties to obtain IDs.  

This tutorial follows the **MicroStratety REST API with Python** examples here https://github.com/tatwan/MicroStratety-Python-REST-API and my previous blog on **How to Embed MicroStrategy Dossier with Jupyter Notebook** here http://www.tarekatwan.com/index.php/2018/01/how-to-embed-microstrategy-dossier-with-jupyter-notebook/

In [2]:
import requests
import json
from pandas.io.json import json_normalize
import pandas as pd

### Create required parameters

In [3]:
### Parameters ###
username = 'Administrator'
password = ''
baseURL = "http://dub-tatwan10.corp.microstrategy.com:8080/MicroStrategyLibrary/api/"

Loading the python functions/script from the tutorial https://github.com/tatwan/MicroStratety-Python-REST-API by placing the code into a `mstr.py` file.

In [4]:
import mstr 

**Steps:**
1. First we login to get `authToken` and a `sessionId` using our `login()` function
2. Once we are authenticated we can list all the projects that we have access to using our `listProjects()` function
3. We save the project ID of the project we are interested in into variable we will name `projectId`  
4. We get the library using our `getLibrary()` function in order to find the Id of the published Dossier we are interested in. In this case we need the `id` from the `target` column and we will save it into `libraryId` variable

In [5]:
#step 1 - authenticate 
authToken, sessionId = mstr.login(baseURL, username, password)

Token: 8fa2ngoiak50597n968cpo7so7
Session ID: {'JSESSIONID': '2E05323BE41684698BF62C35AE9900BB'}


In [6]:
#step 2 - search  projects
projectList = mstr.listProjects(baseURL, authToken, sessionId)
projectList

,id,name,description,status
0,B19DEDCC11D4E0EFC000EB9495D0F44F,MicroStrategy Tutorial,MicroStrategy Tutorial project and application...,0
1,AF09B3E3458F78B4FBE4DEB68528BF7B,Human Resources Analysis Module,The Human Resources Analysis Module analyses w...,0
2,4DD3B04B40D227471401609D630C76ED,Enterprise Manager,,0


In [7]:
#step 3 - Get the project ID of the project we are interested in 
projectId = projectList.iloc[0][0]
projectId

'B19DEDCC11D4E0EFC000EB9495D0F44F'

In [8]:
# step 4 - Get the library List
libraryList = mstr.getLibrary(baseURL, authToken, sessionId, 'DEFAULT')
libraryList

,id,name,projectId,active,lastViewedTime,target
0,21A521BA4DB47ADAEBE19E9E9F7EC7D9,Executive Business User Data Dossier,B19DEDCC11D4E0EFC000EB9495D0F44F,True,2018-08-10T19:36:00.000+0000,FC6E8B6F4950540FC3595093E0FBA306
1,80AFEAD447DE2430F7E41FBB1B1EFCBA,Category Breakdown Dossier,B19DEDCC11D4E0EFC000EB9495D0F44F,True,2018-08-10T18:55:49.000+0000,95005DFF4C4829CF5EE6E98877726566


In [9]:
#Get the Target ID for the published Dossier we want to embed
libraryId = libraryList.iloc[0][5]
libraryId

'FC6E8B6F4950540FC3595093E0FBA306'

Using iPython magic cell we use `%%html` to specifiy were we want to display our Dossier.  

Dossier will be displayed right below this `%5html` cell once the `%%javascript` cell is executed. 

In [18]:
%%html
<script type="text/javascript" src="http://dub-tatwan10.corp.microstrategy.com:8080/MicroStrategyLibrary/javascript/embeddinglib.js"></script>
<div id="dossier1"></div>

Again, using another iPython magic cell with `%%javascript` to load our Dossier 

In [19]:
%%javascript
// we copy the projectID and libraryId and use them for the JavaScript variables below
var projectId = 'B19DEDCC11D4E0EFC000EB9495D0F44F' //IPython.notebook.kernel.execute(projectId);
var libraryId = 'FC6E8B6F4950540FC3595093E0FBA306' //IPython.notebook.kernel.execute(libraryId);

var container = document.getElementById("dossier1"),
 
    url = "http://dub-tatwan10.corp.microstrategy.com:8080/MicroStrategyLibrary/app/" + projectId + '/' + libraryId;
 
    microstrategy.dossier.create({
 
          url: url,
 
          enableResponsive: true,
 
          placeholder: container
       })


<IPython.core.display.Javascript object>